<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/temporal-analysis/part_time_work/analyse_LA_mobility_poverty_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 2.06 ms (started: 2021-04-22 19:04:24 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 40.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=fc226ba388632a3c1d558878c47446bac90bf9fe25b2d868a78429352b85f3ee
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 36.5 MB of archives.
After this 

In [3]:
# Import modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 105 ms (started: 2021-04-22 19:05:20 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 321 ms (started: 2021-04-22 19:05:20 +00:00)


In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 5.07 s (started: 2021-04-22 19:05:20 +00:00)


In [6]:
#sc.stop()

time: 2.84 ms (started: 2021-04-22 19:05:25 +00:00)


In [7]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 22.7 s (started: 2021-04-22 19:05:25 +00:00)


In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'la'

time: 5.86 ms (started: 2021-04-22 19:05:48 +00:00)


In [9]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [10]:
df_soc.createOrReplaceTempView('clean_la')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, part_time_work_behavior_devices FROM clean_la WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+-------------------------------+
|         cbg|                date|device_count|part_time_work_behavior_devices|
+------------+--------------------+------------+-------------------------------+
|060371831013|2020-02-07T00:00:...|          45|                              1|
|060374034022|2020-02-07T00:00:...|         155|                             12|
|060375326041|2020-02-07T00:00:...|          31|                              6|
|060375544031|2020-02-07T00:00:...|         130|                             16|
|060375718003|2020-02-07T00:00:...|          35|                              4|
|060590018024|2020-02-07T00:00:...|          38|                              7|
|060590320463|2020-02-07T00:00:...|         184|                             38|
|060590626281|2020-02-07T00:00:...|         108|                              9|
|060371374011|2020-02-07T00:00:...|          72|                              3|
|060372398012|2020-02-07T00:

In [11]:
df_soc = df_soc.withColumn("date",
    df_soc['date'].substr(0, 10))

time: 67.7 ms (started: 2021-04-22 19:06:21 +00:00)


In [12]:
df_soc.show()

+------------+----------+------------+-------------------------------+
|         cbg|      date|device_count|part_time_work_behavior_devices|
+------------+----------+------------+-------------------------------+
|060371831013|2020-02-07|          45|                              1|
|060374034022|2020-02-07|         155|                             12|
|060375326041|2020-02-07|          31|                              6|
|060375544031|2020-02-07|         130|                             16|
|060375718003|2020-02-07|          35|                              4|
|060590018024|2020-02-07|          38|                              7|
|060590320463|2020-02-07|         184|                             38|
|060590626281|2020-02-07|         108|                              9|
|060371374011|2020-02-07|          72|                              3|
|060372398012|2020-02-07|          41|                              1|
|060373009011|2020-02-07|          82|                             16|
|06037

In [13]:
# Add completely home percentage column
#df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
#df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

#df_soc.show()

time: 74.1 ms (started: 2021-04-22 19:06:21 +00:00)


In [14]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 36.1 ms (started: 2021-04-22 19:06:21 +00:00)




---


**Demographic data**





---



In [15]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'la.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|060373104003|     1286|17.884914463452567| 69.51788491446345|
|060590865023|     2545|22.789783889980352| 85.34381139489194|
|060376510024|     1285|               0.0|42.490272373540854|
|060376511011|     2708| 7.745398773006134| 50.70162481536189|
|060590994023|      672|12.969283276450511| 38.54166666666667|
|060590995094|      342| 17.83625730994152| 77.48538011695906|
|060376511012|     2483|2.6178010471204187|50.302053966975436|
|060376512011|      859|2.8846153846153846| 82.42142025611176|
|060376512012|     1179| 1.441899915182358|  74.1306191687871|
|060376512014|      989|2.4266936299292214| 70.57633973710819|
|060376512211|     3373|  9.95684340320592| 40.61666172546695|
|060590995092|      938| 5.756929637526652|  84.9680170575693|
|060590995095|      576|14.583333333333334|           7

In [16]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+----------+------------+-------------------------------+-------------------------+------------------+
|         cbg|      date|device_count|part_time_work_behavior_devices|part_time_work_percentage|poverty_percentage|
+------------+----------+------------+-------------------------------+-------------------------+------------------+
|060371831013|2020-02-07|          45|                              1|       2.2222222222222223|15.450643776824036|
|060374034022|2020-02-07|         155|                             12|        7.741935483870968| 10.67193675889328|
|060375326041|2020-02-07|          31|                              6|        19.35483870967742| 27.79187817258883|
|060375544031|2020-02-07|         130|                             16|       12.307692307692308|  25.3447633246366|
|060375718003|2020-02-07|          35|                              4|       11.428571428571429|3.1537450722733245|
|060590018024|2020-02-07|          38|                              7|  

In [17]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("part_time_work_percentage").withColumnRenamed('avg(part_time_work_percentage)','part_time_work_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("part_time_work_percentage").withColumnRenamed('avg(part_time_work_percentage)','part_time_work_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.part_time_work_percentage as top20_perc,
l.part_time_work_percentage as bottom20_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")


time: 332 ms (started: 2021-04-22 19:06:24 +00:00)


**Load 2019 stats**

In [18]:
filename = '{}_social_summary_2019.csv'.format(city)
stat_2019 = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/filename))
stat_2019.show()
mean_2019 = stat_2019.collect()[1][6]


+-----+------------------+-----------------+-----------------+-----------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+------------------+
|  _c0|             total|  completely_home|   part_time_work|   full_time_work|percentage_completely_home|percentage_part_time_work|percentage_full_time_work|norm_completely_home| norm_part_time_work| norm_full_time_work|             month|
+-----+------------------+-----------------+-----------------+-----------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+------------------+
|count|         3004769.0|        3004769.0|        3004769.0|        3004769.0|                 3004769.0|                3004769.0|                3004769.0|           3004769.0|           3004769.0|           3004769.0|         3004769.0|
| mean| 87.84796368705881|27.763

In [19]:
# calculate deviation form mean
grouped_df_mob_demo_combined= grouped_df_mob_demo_combined.withColumn("top20_deviation",
    grouped_df_mob_demo_combined['top20_perc'] - mean_2019)


time: 48.9 ms (started: 2021-04-22 19:06:25 +00:00)


In [20]:
grouped_df_mob_demo_combined = grouped_df_mob_demo_combined.withColumn("bottom20_deviation",
    grouped_df_mob_demo_combined['bottom20_perc'] - mean_2019)

time: 41.2 ms (started: 2021-04-22 19:06:25 +00:00)



**Visualizations** 



In [21]:
# create altair library theme 

def my_theme():
  return {
    'config': {
      'view': {'continuousHeight': 300, 'continuousWidth': 400},  # from the default theme
      'range': {'category': {'scheme': 'redblue'}}##['#FF4500','#a0aab4','#4E79A7']}}
    }
  }
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')
# 

ThemeRegistry.enable('my_theme')

time: 13 ms (started: 2021-04-22 19:06:25 +00:00)


In [22]:
def create_chart(df) :
  chart = alt.Chart(df).transform_fold(
    fold=['Bottom 20 % Deviation', 'Top 20 % Deviation'],
    as_ = ['Poverty %','part_time_value']
    
    ).mark_line().encode(
      x= alt.X('date:T', title = 'Date'),
      y= alt.Y('part_time_value:Q', title = 'Part Time Work %'),
      color='Poverty %:N'
    ).properties(width = 1000)

  rule = alt.Chart(pd.DataFrame({'Mean 2019':[0]})).mark_rule(color='#757575',strokeDash=[5,3], size=2).encode(
    y='Mean 2019',
    size=alt.value(2),
    )
  
  text = alt.Chart({'values':[{ 'y': 1}]}).mark_text(
    text='Mean 2019', angle=0
    ).encode(
      y=alt.Y('y:Q'),opacity=alt.value(0.4)
    )

  return  chart + rule + text


time: 31.7 ms (started: 2021-04-22 19:06:25 +00:00)


In [23]:
def apply_date_filter(df,start_date,end_date) :
  mask = (df['date'] > start_date) & (df['date'] <= end_date)
  dfX = df.loc[mask]
  return dfX

time: 10.4 ms (started: 2021-04-22 19:06:25 +00:00)


In [24]:
# create pandas dataframe for visualizations
df = grouped_df_mob_demo_combined.toPandas()
df

,date,top20_perc,bottom20_perc,top20_deviation,bottom20_deviation
0,2020-02-26,13.151697,7.146811,3.546865,-2.458021
1,2020-04-13,5.511743,4.146589,-4.093089,-5.458243
2,2020-06-24,6.510721,4.887808,-3.094111,-4.717024
3,2020-06-08,7.115684,3.949645,-2.489148,-5.655187
4,2020-09-12,6.838127,6.162825,-2.766705,-3.442007
...,...,...,...,...,...
361,2020-04-05,3.346791,3.603771,-6.258041,-6.001061
362,2020-05-01,5.550260,3.083870,-4.054572,-6.520962
363,2020-10-25,6.236403,11.772826,-3.368429,2.167994
364,2020-12-28,7.855217,7.590284,-1.749615,-2.014548


time: 4min 34s (started: 2021-04-22 19:06:25 +00:00)


In [25]:
df = df.rename(columns={'top20_perc': 'Top 20 %', 'bottom20_perc': 'Bottom 20 %','top20_deviation': 'Top 20 % Deviation', 'bottom20_deviation': 'Bottom 20 % Deviation'})

time: 3.55 ms (started: 2021-04-22 19:11:00 +00:00)


In [26]:
# analysis for the entire year on monthly basis
df_str = df.copy(deep=True) #make a deep copy

df_str['date'] = pd.to_datetime(df_str['date'],utc= True) # extract the months

df_str = df_str.groupby([df_str['date']]).mean().rolling(10).mean()[0:] #find the mean for the days and roll up for every 10 days

chart = alt.Chart(df_str.reset_index()).transform_fold(
    fold=['Bottom 20 %', 'Top 20 %'],
    as_ = ['Poverty %','part_time_value']
    
    ).mark_line().encode(
      x='date:T',
      y= alt.Y('part_time_value:Q', title = 'Part Time %'), color='Poverty %:N'
    ).properties(width = 1000)
chart

alt.Chart(...)

time: 161 ms (started: 2021-04-22 19:11:00 +00:00)


In [27]:
#Conversion of datetime format
df['date'] = pd.to_datetime(df['date'],utc= True)
#df['date'] = [datetime.datetime.date(d) for d in df['date']] 

time: 6.54 ms (started: 2021-04-22 19:11:00 +00:00)


Filtering based on important dates



*   Thursday, March 19, 2020: Statewide Stay-at-Home Order Issued











In [28]:
start_date = '2020-3-16'
end_date = '2020-4-9'
df1 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df1)
chart

alt.LayerChart(...)

time: 109 ms (started: 2021-04-22 19:11:00 +00:00)





*   August. 7, 2020: LA County Exceeds 200,000 Coronavirus Cases
*   August, 12, 2020: LA County Crosses 'Tragic Milestone' Of 5,000 Coronavirus Deaths 









In [29]:
start_date = '2020-8-5'
end_date = '2020-8-21'
df2 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df2)
chart
#df2

alt.LayerChart(...)

time: 83.9 ms (started: 2021-04-22 19:11:00 +00:00)


*   Oct. 14, 2020: LA County Sees Rise In Workplace Outbreaks As Infection Rate Creeps Upward





In [30]:
start_date = '2020-10-12'
end_date = '2020-10-24'
df3 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df3)
chart

alt.LayerChart(...)

time: 74.2 ms (started: 2021-04-22 19:11:00 +00:00)


Dec. 16, 2020:
LA County COVID-19 Deaths Hit New Record


In [31]:
start_date = '2020-12-15'
end_date = '2020-12-31'
df4 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df4)
chart

alt.LayerChart(...)

time: 71.2 ms (started: 2021-04-22 19:11:00 +00:00)
